In [12]:
#Used Libraries
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [13]:
#1. What is the position played by the majority of the squad?

queryResult("""
SELECT
    position,
    COUNT(DISTINCT(player_id))
FROM
    squad
GROUP BY
    position
ORDER BY
    COUNT(DISTINCT(player_id)) DESC;
""")


,position,count
0,df,9
1,fw,8
2,mf,8
3,gk,4


The majority of the squad are defenders (df position)

In [14]:
#2. Which players played the most games during the entire season?

queryResult("""
SELECT
    squad.first_name,
    squad.last_name,
    games_played.player_id,
    games_played.league,
    games_played.champions,
    games_played.cup,
    games_played.supercup,
    games_played.league + games_played.champions + games_played.cup + games_played.supercup AS total
FROM
    games_played INNER JOIN squad ON games_played.player_id = squad.player_id
ORDER BY
    total DESC
LIMIT 5;
""")


,first_name,last_name,player_id,league,champions,cup,supercup,total
0,Pedro,Lopez,19,37,7,6,2,52
1,Frenkie,De Jong,15,37,7,5,2,51
2,Antoine,Griezmann,28,36,7,6,2,51
3,Sergio,Busquets,14,36,6,6,2,50
4,Jordi,Alba,12,35,7,5,2,49


Pedro Lopez(19) played the most games which is 52 

In [15]:
#3. Out of the players who played less than 1000 minutes in the league who scored the most goals?

queryResult("""
SELECT
    squad.first_name,
    squad.last_name,
    squad.player_id,
    league_stats.league_minutes,
    league_goals
FROM
    squad INNER JOIN league_stats ON squad.player_id = league_stats.player_id
WHERE
    league_stats.league_minutes <1000
ORDER BY
    league_stats.league_goals DESC
LIMIT 5;
""")


,first_name,last_name,player_id,league_minutes,league_goals
0,Ansu,Fati,22,435,4
1,Francisco,Trincão,25,683,3
2,Philippe,Coutinho,18,656,2
3,Sergi,Roberto,13,943,1
4,Junior,Firpo,5,285,1


Ansu Fati(22) scored the most goals out of the players who played less then 100 minutes

In [16]:
#4. Which players played the most minutes in the league? Show top 5.

queryResult("""
SELECT
    squad.first_name,
    squad.last_name,
    squad.player_id,
    league_stats.league_minutes,
    league_goals
FROM
    squad INNER JOIN league_stats ON squad.player_id = league_stats.player_id
ORDER BY
    league_stats.league_minutes DESC
LIMIT 5;
""")




,first_name,last_name,player_id,league_minutes,league_goals
0,Frenkie,De Jong,15,3158,3
1,Jordi,Alba,12,3030,3
2,Lionel,Messi,29,3022,30
3,Marc-André,Ter Stegen,1,2790,0
4,Antoine,Griezmann,28,2619,13


In [17]:
#5.How many assists were made by the forwards?

queryResult("""
SELECT
    squad.position,
    SUM(league_stats.league_assists),
    SUM(champions_stats.champions_assists),
    SUM(cup_stats.cup_assists),
    SUM(supercup_stats.supercup_assists),
    SUM(league_stats.league_assists+champions_stats.champions_assists+cup_stats.cup_assists+supercup_stats.supercup_assists) AS total
FROM
    squad INNER JOIN league_stats ON squad.player_id = league_stats.player_id
    INNER JOIN champions_stats ON squad.player_id = champions_stats.player_id
    INNER JOIN cup_stats ON squad.player_id = cup_stats.player_id
    INNER JOIN supercup_stats ON squad.player_id = supercup_stats.player_id
GROUP BY
    squad.position
ORDER BY
    total DESC;
""")

,position,sum,sum,sum,sum,total
0,fw,27,9,5,1,42
1,mf,17,3,4,0,24
2,df,14,3,5,1,23
3,gk,0,0,0,0,0


42

In [18]:
#6. Who are the top 3 players with the best goals per minute record? (clarification: how many minutes pass between each goals. For example if a player scored 5 goals and played 600 minutes then his goals per minute record is a goal every 120 minutes)

queryResult("""
SELECT
    squad.first_name,
    squad.last_name,
    squad.player_id,
    SUM(league_stats.league_minutes+champions_stats.champions_minutes+cup_stats.cup_minutes+supercup_stats.supercup_minutes) AS total_minutes,
    SUM(league_stats.league_goals+champions_stats.champions_goals+cup_stats.cup_goals+supercup_stats.supercup_goals) AS total_goals, 
    cast(SUM(league_stats.league_minutes+champions_stats.champions_minutes+cup_stats.cup_minutes+supercup_stats.supercup_minutes)/SUM(league_stats.league_goals+champions_stats.champions_goals+cup_stats.cup_goals+supercup_stats.supercup_goals) as decimal(10,2)) as goals_per_minute
FROM
    squad INNER JOIN league_stats ON squad.player_id = league_stats.player_id
    INNER JOIN champions_stats ON squad.player_id = champions_stats.player_id
    INNER JOIN cup_stats ON squad.player_id = cup_stats.player_id
    INNER JOIN supercup_stats ON squad.player_id = supercup_stats.player_id
WHERE 
    league_stats.league_goals+champions_stats.champions_goals+cup_stats.cup_goals+supercup_stats.supercup_goals >0
GROUP BY
    squad.first_name,
    squad.last_name,
    squad.player_id
ORDER BY
    goals_per_minute
LIMIT 3;
""")


,first_name,last_name,player_id,total_minutes,total_goals,goals_per_minute
0,Lionel,Messi,29,4192,38,110.0
1,Ansu,Fati,22,596,5,119.0
2,Antoine,Griezmann,28,3903,20,195.0


In [19]:
#7. How many players did not contribute goals or assists in the league?

queryResult("""
SELECT
    squad.first_name,
    squad.last_name,
    squad.player_id,
    SUM(league_stats.league_goals) AS sum_goals,
    SUM(league_stats.league_assists) AS sum_asissts
FROM
    squad INNER JOIN league_stats ON squad.player_id = league_stats.player_id
WHERE
    league_stats.league_goals = 0 AND league_stats.league_assists = 0
GROUP BY
    squad.first_name,
    squad.last_name,
    squad.player_id;
""")



,first_name,last_name,player_id,sum_goals,sum_asissts
0,Arnau,Tenas,4,0,0
1,Gerard,Piqué,7,0,0
2,Iñaki,Peña,3,0,0
3,Konard,De La Fuente,23,0,0
4,Marc-André,Ter Stegen,1,0,0
5,Matheus,Fernandes,21,0,0
6,Miralem,Pjanić,16,0,0
7,Norberto,Murara,2,0,0
8,Samuel,Umtiti,10,0,0
9,Álex,Collado,26,0,0


10 players

In [20]:
#8. Who is the player that has the most yellow cards in the league? (If there is a tie then sort by minutes played)

queryResult("""
SELECT
    squad.first_name,
    squad.last_name,
    squad.player_id,
    SUM(league_stats.league_yellow) AS sum_yellow,
    SUM(league_stats.league_minutes) AS sum_minutes
FROM
    squad INNER JOIN league_stats ON squad.player_id = league_stats.player_id
GROUP BY
    squad.first_name,
    squad.last_name,
    squad.player_id
ORDER BY
    sum_yellow DESC,
    sum_minutes
LIMIT 5;
""")


,first_name,last_name,player_id,sum_yellow,sum_minutes
0,Sergio,Busquets,14,9,2527
1,Jordi,Alba,12,9,3030
2,Clément,Lenglet,9,7,2476
3,Frenkie,De Jong,15,5,3158
4,Óscar,Mingueza,11,4,1901


Sergio Busquets, it's a tie between him and Jordi Alba, but Sergio Busquets played less minutes.

In [21]:
#9. Which defender contribute the most goals and assists in the league?

queryResult("""
SELECT
    squad.first_name,
    squad.last_name,
    squad.player_id,
    squad.position,
    SUM(league_stats.league_goals) AS sum_goals,
    SUM(league_stats.league_assists) AS sum_asissts,
    SUM(league_stats.league_goals+league_stats.league_assists) AS total_goals_assists
FROM
    squad INNER JOIN league_stats ON squad.player_id = league_stats.player_id
WHERE
    squad.position = 'df'
GROUP BY
    squad.first_name,
    squad.last_name,
    squad.player_id,
    squad.position
ORDER BY
    total_goals_assists DESC
LIMIT 5;
""")



,first_name,last_name,player_id,position,sum_goals,sum_asissts,total_goals_assists
0,Jordi,Alba,12,df,3,7,10
1,Óscar,Mingueza,11,df,2,2,4
2,Sergi,Roberto,13,df,1,2,3
3,Sergiño,Dest,6,df,2,1,3
4,Ronald,Araújo,8,df,2,1,3


Jordi Alba

In [22]:
#10. How many players played more then 50 hours total in the league and in the champions league?

queryResult("""
SELECT
    squad.first_name,
    squad.last_name,
    squad.player_id,
    SUM(league_stats.league_minutes+champions_stats.champions_minutes)/60 AS total_hours
FROM
    squad INNER JOIN league_stats ON squad.player_id = league_stats.player_id
    INNER JOIN champions_stats ON squad.player_id = champions_stats.player_id
GROUP BY
    squad.first_name,
    squad.last_name,
    squad.player_id
ORDER BY
    total_hours DESC
LIMIT 10
""")

,first_name,last_name,player_id,total_hours
0,Frenkie,De Jong,15,62
1,Lionel,Messi,29,59
2,Jordi,Alba,12,58
3,Marc-André,Ter Stegen,1,54
4,Antoine,Griezmann,28,52
5,Clément,Lenglet,9,50
6,Pedro,Lopez,19,48
7,Sergio,Busquets,14,47
8,Sergiño,Dest,6,41
9,Óscar,Mingueza,11,35


In [23]:
query = '''
        SELECT 
            COUNT(((cs.champions_minutes + ls.league_minutes) / 60.0))  AS num_of_player_over_3k_minutes
        FROM
            squad s
        JOIN champions_stats cs ON s.player_id = cs.player_id
        JOIN league_stats ls ON s.player_id = ls.player_id
        WHERE ((cs.champions_minutes + ls.league_minutes) / 60.0) > 50
        ORDER BY
            num_of_player_over_3k_minutes DESC;
'''
queryResult(query)

,num_of_player_over_3k_minutes
0,6


6 players